In [1]:
import numpy as np

import mlgrad.model as model
import mlgrad.loss as loss
import mlgrad.func as func

from mlgrad import erm_fg, erisk

from pprint import pprint

from axon import loads, dumps

In [2]:
X = np.array([[-1,-1],[-1,1],[1,-1],[1,1]], 'd')
Y = np.array([-1,1,1,-1],'d')

In [3]:
nn = model.FFNetworkModel()
layer = model.ModelLayer(2)
layer.add(model.SigmaNeuronModel(func.Sigmoidal(), 3))
layer.add(model.SigmaNeuronModel(func.Sigmoidal(), 3))
#layer.add(model.SigmaNeuronModel(func.Sigmoidal(), 3))
print(layer.n_param)
nn.add(layer)

6


In [4]:
ffnn = model.FFNetworkFuncModel(model.LinearModel(3), nn)
ffnn.allocate()
print(ffnn.n_param)
ffnn.param[:] = np.random.random(size=ffnn.n_param)
print(list(ffnn.param))

9
[0.05708447872343059, 0.09481125054835982, 0.3546848426756265, 0.519296455393528, 0.3259496988028895, 0.01812231488524363, 0.7974450396896586, 0.6731611434264447, 0.7827638081852911]


In [5]:
print(list(ffnn.head.param))
print(list(ffnn.body.param))

[0.05708447872343059, 0.09481125054835982, 0.3546848426756265]
[0.519296455393528, 0.3259496988028895, 0.01812231488524363, 0.7974450396896586, 0.6731611434264447, 0.7827638081852911]


In [6]:
for x, y in zip(X,Y):
    print(x,ffnn(x.copy()),y)

[-1. -1.] 0.22960913971394942 -1.0
[-1.  1.] 0.36215985946804763 1.0
[ 1. -1.] 0.35908714259742935 1.0
[ 1.  1.] 0.4447400888063039 -1.0


In [7]:
loss_func = loss.ErrorLoss(func.Square())
er = erisk(X, Y, ffnn, loss_func)
fg = erm_fg(er, h=0.01, tol=1.0e-9)

418 [-1.07582727 -2.23437528  2.28114515 -3.08779404  3.67680517  3.73063368
  3.00440253  3.16143353  3.21040044]


In [8]:
for x, y in zip(X,Y):
    print(x,ffnn(x),y)

[-1. -1.] -0.9998608896055221 -1.0
[-1.  1.] 0.9999106898129984 1.0
[ 1. -1.] 0.9998925350012222 1.0
[ 1.  1.] -0.9999134682518958 -1.0


In [9]:
print(dumps(ffnn.as_json(), pretty=1))

layers: [[ { param: (-3.0877940438741263 3.6768051727528697 3.7306336751939475)
      type: "Σ-neuron"}
    { param: (3.004402531313504 3.161433532239766 3.2104004383240614)
      type: "Σ-neuron"}]]
head: {
  param: (-1.0758272749485145 -2.2343752807383295 2.281145145988444)
  type: "linear"}


In [10]:
ffnn([-1.,-1.])

-0.9998608896055221

In [11]:
nn = model.FFNetworkModel()
layer = model.ModelLayer(2)
layer.add(model.SigmaNeuronModel(func.Sigmoidal(), 3))
layer.add(model.SigmaNeuronModel(func.Sigmoidal(), 3))
layer.add(model.SigmaNeuronModel(func.Sigmoidal(), 3))
nn.add(layer)

In [12]:
ffnn = model.FFNetworkFuncModel(model.LinearModel(4), nn)
ffnn.allocate()
print(ffnn.n_param)
ffnn.param[:] = np.random.random(size=ffnn.n_param)
print(ffnn.param.base)

13
[ 0.8467467   0.67407487  0.20955524  0.04400143  0.68814888  0.63069438
  0.69050834  0.44656649  0.71770948  0.33377161  0.62263056  0.63122931
  0.40743911]


In [13]:
loss_func = loss.MarginLoss(func.Hinge())
er = erisk(X, Y, ffnn, loss_func)
fg = erm_fg(er, h=0.1, tol=1.0e-9)

35 [-0.14528603  1.64988025 -2.96134617 -1.78950968  2.74830011  3.30307843
  3.55893574 -3.38983881  4.20246344  3.98174766 -2.72422271 -2.84880959
 -2.7554257 ]


In [15]:
for x, y in zip(X,Y):
    print(x,np.sign(ffnn(x)),y)

[-1. -1.] -1.0 -1.0
[-1.  1.] 1.0 1.0
[ 1. -1.] 1.0 1.0
[ 1.  1.] -1.0 -1.0
